In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('swat.csv')
#df.shape

df = df.iloc[:5*len(df.index)//8]
#df = df.iloc[:len(df.index)//2]
#df = df.iloc[:7500]
y = df["Normal/Attack"]
del df["Normal/Attack"]
df.head(5)

,timestamp,FIT101,LIT101,MV101,P101,P102,AIT201,AIT202,AIT203,FIT201,...,FIT504,P501,P502,PIT501,PIT502,PIT503,FIT601,P601,P602,P603
0,1451296800000000000,2.427057,522.8467,2,2,1,262.0161,8.396437,328.6337,2.445391,...,0.307786,2,1,250.8652,1.649953,189.5988,0.000128,1,1,1
1,1451296801000000000,2.446274,522.8860,2,2,1,262.0161,8.396437,328.6337,2.445391,...,0.307786,2,1,250.8652,1.649953,189.6789,0.000128,1,1,1
2,1451296802000000000,2.489191,522.8467,2,2,1,262.0161,8.394514,328.6337,2.442316,...,0.308619,2,1,250.8812,1.649953,189.6789,0.000128,1,1,1
3,1451296803000000000,2.534350,522.9645,2,2,1,262.0161,8.394514,328.6337,2.442316,...,0.308619,2,1,250.8812,1.649953,189.6148,0.000128,1,1,1
4,1451296804000000000,2.569260,523.4748,2,2,1,262.0161,8.394514,328.6337,2.443085,...,0.308619,2,1,250.8812,1.649953,189.5027,0.000128,1,1,1


In [3]:
y = np.array(y, dtype=str)
y_binary = []
for i in y:
    if i == "Normal":
        y_binary.append(False)  #false == normal
    else:
        y_binary.append(True)   #true == attack
        
y_binary = np.array(y_binary)
y_binary

array([False, False, False, ...,  True,  True,  True])

In [4]:
feature_names = np.array(df.columns, dtype=str)
feature_names = feature_names[1:]

In [5]:
from sklearn.preprocessing import StandardScaler
# Separating out the features
x = df.loc[:, feature_names].values
x_scl = StandardScaler().fit_transform(x)

df_scl = pd.DataFrame(data = x_scl)
df_scl.head(5)

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,0.654471,-0.762548,0.748584,0.715878,-0.104298,1.08142,-0.700522,-0.146441,0.708517,0.709406,...,0.371398,0.365016,0.0,0.384298,1.122849,0.390325,-0.100061,0.0,-0.092879,0.0
1,0.670213,-0.762240,0.748584,0.715878,-0.104298,1.08142,-0.700522,-0.146441,0.708517,0.709406,...,0.371398,0.365016,0.0,0.384298,1.122849,0.391693,-0.100061,0.0,-0.092879,0.0
2,0.705369,-0.762548,0.748584,0.715878,-0.104298,1.08142,-0.719069,-0.146441,0.705867,0.709406,...,0.379814,0.365016,0.0,0.384508,1.122849,0.391693,-0.100061,0.0,-0.092879,0.0
3,0.742362,-0.761624,0.748584,0.715878,-0.104298,1.08142,-0.719069,-0.146441,0.705867,0.709406,...,0.379814,0.365016,0.0,0.384508,1.122849,0.390598,-0.100061,0.0,-0.092879,0.0
4,0.770960,-0.757619,0.748584,0.715878,-0.104298,1.08142,-0.719069,-0.146441,0.706530,0.709406,...,0.379814,0.365016,0.0,0.384508,1.122849,0.388684,-0.100061,0.0,-0.092879,0.0


In [6]:
def cor_selector(X, y,num_feats):
    cor_list = []
    feature_name = X.columns.tolist()
    # calculate the correlation with y for each feature
    for i in X.columns.tolist():
        cor = np.corrcoef(X[i], y)[0, 1]
        cor_list.append(cor)
    # replace NaN with 0
    cor_list = [0 if np.isnan(i) else i for i in cor_list]
    # feature name
    cor_feature = X.iloc[:,np.argsort(np.abs(cor_list))[-num_feats:]].columns.tolist()
    # feature selection? 0 for not select, 1 for select
    cor_support = [True if i in cor_feature else False for i in feature_name]
    return cor_support, cor_feature
cor_support, cor_feature = cor_selector(df_scl, y_binary, len(feature_names))
print(str(len(cor_feature)), 'selected features')
print(str(len(cor_support)), 'cor_support')

51 selected features
51 cor_support


/home/top/.pyenv/versions/3.7.12/lib/python3.7/site-packages/numpy/lib/function_base.py:2530: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/top/.pyenv/versions/3.7.12/lib/python3.7/site-packages/numpy/lib/function_base.py:2531: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [7]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler
X_norm = MinMaxScaler().fit_transform(df_scl)
chi_selector = SelectKBest(chi2, k=len(feature_names))
chi_selector.fit(X_norm, y_binary)
chi_support = chi_selector.get_support()
chi_feature = df_scl.loc[:,chi_support].columns.tolist()
print(str(len(chi_feature)), 'selected features')
print(str(len(chi_support)), 'chi_support')

51 selected features
51 chi_support


In [8]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
rfe_selector = RFE(estimator=LogisticRegression(solver='lbfgs', multi_class='auto', max_iter=1000), n_features_to_select=len(feature_names), step=10, verbose=5)
rfe_selector.fit(X_norm, y_binary)
rfe_support = rfe_selector.get_support()
rfe_feature = df_scl.loc[:,rfe_support].columns.tolist()
print(str(len(rfe_feature)), 'selected features')
print(str(len(rfe_support)), 'rfe_support')

51 selected features
51 rfe_support


In [9]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression

embeded_lr_selector = SelectFromModel(LogisticRegression(solver='lbfgs', multi_class='auto', penalty="l2", max_iter=1000), max_features=len(feature_names))
embeded_lr_selector.fit(X_norm, y_binary)

embeded_lr_support = embeded_lr_selector.get_support()
embeded_lr_feature = df_scl.loc[:,embeded_lr_support].columns.tolist()
print(str(len(embeded_lr_feature)), 'selected features')
print(str(len(embeded_lr_support)), 'embeded_lr_support')

19 selected features
51 embeded_lr_support


In [10]:
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier

embeded_rf_selector = SelectFromModel(RandomForestClassifier(n_estimators=100), max_features=len(feature_names))
embeded_rf_selector.fit(df_scl, y_binary)

embeded_rf_support = embeded_rf_selector.get_support()
embeded_rf_feature = df_scl.loc[:,embeded_rf_support].columns.tolist()
print(str(len(embeded_rf_feature)), 'selected features')
print(str(len(embeded_rf_support)), 'embeded_rf_support')

19 selected features
51 embeded_rf_support


In [11]:
# put all selection together
feature_selection_df = pd.DataFrame({'Feature':feature_names, 'Pearson':cor_support, 'Chi-2':chi_support, 'RFE':rfe_support, 'Logistics':embeded_lr_support,
                                    'Random Forest':embeded_rf_support})
# count the selected times for each feature
feature_selection_df['Total'] = np.sum(feature_selection_df, axis=1)
# display the top 100
feature_selection_df = feature_selection_df.sort_values(['Total','Feature'] , ascending=False)
feature_selection_df.index = range(1, len(feature_selection_df)+1)
feature_selection_df.head(len(feature_names))

,Feature,Pearson,Chi-2,RFE,Logistics,Random Forest,Total
1,PIT503,True,True,True,True,True,5
2,PIT501,True,True,True,True,True,5
3,LIT101,True,True,True,True,True,5
4,FIT504,True,True,True,True,True,5
5,FIT502,True,True,True,True,True,5
6,FIT501,True,True,True,True,True,5
7,FIT401,True,True,True,True,True,5
8,FIT201,True,True,True,True,True,5
9,DPIT301,True,True,True,True,True,5
10,AIT202,True,True,True,True,True,5


In [12]:
print(feature_selection_df)
selected_features = []
for i in range(1, len(feature_selection_df["Total"])):
    print(feature_selection_df["Feature"][i])
    print(feature_selection_df["Total"][i])
    if feature_selection_df["Total"][i] > 3:
        selected_features.append(feature_selection_df["Feature"][i])

print(selected_features)

    Feature  Pearson  Chi-2   RFE  Logistics  Random Forest  Total
1    PIT503     True   True  True       True           True      5
2    PIT501     True   True  True       True           True      5
3    LIT101     True   True  True       True           True      5
4    FIT504     True   True  True       True           True      5
5    FIT502     True   True  True       True           True      5
6    FIT501     True   True  True       True           True      5
7    FIT401     True   True  True       True           True      5
8    FIT201     True   True  True       True           True      5
9   DPIT301     True   True  True       True           True      5
10   AIT202     True   True  True       True           True      5
11    UV401     True   True  True      False           True      4
12   PIT502     True   True  True      False           True      4
13     P501     True   True  True      False           True      4
14     P403     True   True  True       True          False   

In [13]:
df_fs = df[selected_features]
df_fs.head()

,PIT503,PIT501,LIT101,FIT504,FIT502,FIT501,FIT401,FIT201,DPIT301,AIT202,...,FIT503,AIT504,AIT503,AIT502,AIT501,AIT402,AIT401,AIT203,P201,AIT201
0,189.5988,250.8652,522.8467,0.307786,1.279621,1.723789,1.713517,2.445391,19.74838,8.396437,...,0.735269,12.03538,264.5475,145.1166,7.878621,156.0882,148.808,328.6337,1,262.0161
1,189.6789,250.8652,522.8860,0.307786,1.297554,1.723789,1.715952,2.445391,19.74838,8.396437,...,0.735269,12.03538,264.5475,145.1166,7.878621,156.0882,148.808,328.6337,1,262.0161
2,189.6789,250.8812,522.8467,0.308619,1.293967,1.723404,1.715952,2.442316,19.69076,8.394514,...,0.735269,12.03538,264.5475,145.1166,7.878621,156.0882,148.808,328.6337,1,262.0161
3,189.6148,250.8812,522.9645,0.308619,1.281158,1.723404,1.714670,2.442316,19.69076,8.394514,...,0.735269,12.03538,264.5475,145.0141,7.878621,156.0882,148.808,328.6337,1,262.0161
4,189.5027,250.8812,523.4748,0.308619,1.281158,1.723404,1.714670,2.443085,19.69076,8.394514,...,0.735269,12.03538,264.5475,144.8859,7.878621,156.0882,148.808,328.6337,1,262.0161
